# Taller de Estadística Básica y Análisis de regresión Multilineal

Instalamos e importamos los paquetes de Python que usaremos en los análisis.

In [ ]:
import pandas as pd
import numpy as np

from numpy import int64, float64

import matplotlib.pylab as plt
import matplotlib.pyplot
import seaborn as sns
from plotnine import *
from pyparsing.helpers import line

## Ejercicio 1

Considere los archivos de datos de las páginas del Banco Mundial para Colombia:

<https://data.worldbank.org/indicator/SP.POP.65UP.TO?end=2021&amp;amp;locations=CO&amp;amp;start=1960>

<https://data.worldbank.org/indicator/SP.POP.0014.TO.ZS?end=2021&amp;amp;locations=CO&amp;amp;start=1960>

Primero hacemos una extracción de los datos que vamos a utilizar de los archivos excel obtenidos en las URL. 

In [ ]:
data_poblation_65_above = pd.read_csv('https://raw.githubusercontent.com/AnalisisDeBasesDatos/TallerEstadBasicaRegMultilineal/main/poblation64andabove.csv', skiprows=4 )
data_poblation_0_14 = pd.read_csv('https://raw.githubusercontent.com/AnalisisDeBasesDatos/TallerEstadBasicaRegMultilineal/main/poblation_0_14.csv', skiprows=4 )

In [ ]:
data_poblation_col_65_above = data_poblation_65_above.iloc[[45], 4:66].transpose()
data_poblation_col_65_above = data_poblation_col_65_above.reset_index()
data_poblation_col_65_above.columns = ['year', 'percent']
data_poblation_col_65_above['year'] = data_poblation_col_65_above['year'].astype(int64)
data_poblation_col_65_above['percent'] = data_poblation_col_65_above['percent'].astype(float64)

In [ ]:
data_poblation_col_0_14= data_poblation_0_14.iloc[ [45], 4:66].transpose()
data_poblation_col_0_14 = data_poblation_col_0_14.reset_index()
data_poblation_col_0_14.columns = ['year', 'percent']
data_poblation_col_0_14['year'] = data_poblation_col_0_14['year'].astype(int64)
data_poblation_col_0_14['percent'] = data_poblation_col_0_14['percent'].astype(float64)

Usamos la libreria plotnine para la visualizion de dispersión de los datos.

In [ ]:
(ggplot(data_poblation_col_65_above, aes(x='year', y='percent'))
+ geom_point(fill="#94FC26",colour="#94FC26")
+ ggtitle('Porcentaje Población x edad >= 65 x Año ')
+ labs(x='Año', y='Porcentaje Población'))

In [ ]:
(ggplot(data_poblation_col_0_14, aes(x='year', y='percent'))
+ geom_point(fill="#94FC26",colour="#94FC26")
+ ggtitle('Porcentaje Población x edad 0-14 x Año ')
+ labs(x='Año', y='Porcentaje Población'))

### Use el método de regresión lineal para ajustar una función no lineal o lineal a los siguientes pares de datos:


#### Porcentaje de población total vs. Años para Población con edades ≥ 65 años

Realizaremos una regresión lineal, polinomial y exponencial de los porcentajes de edades >= 65 X Año.
- Regresion Lineal:

In [ ]:
X = data_poblation_col_65_above['year'].values
Y = data_poblation_col_65_above['percent'].values

In [ ]:
a = np.inner(X,X)
b = np.sum(X)
c = np.inner(X,Y)
d = np.sum(Y)
N = len(X)
delta = (a)*(N)-(b**2)
A = (N*c-b*d)/delta
B = (a*d-b*c)/delta
print('B: ', B, "A: ", A)
data_poblation_col_65_above['y_pred_lineal']= A*X+B

- Regresión Exponencial

In [ ]:
Y_log = np.log(Y) #Conversion de los datos y en log(y)
a = np.inner(X,X)
b = np.sum(X)
c = np.inner(X,Y_log)
d = np.sum(Y_log)
N = len(X)
delta = (a)*(N)-(b**2)
A = (N*c-b*d)/delta
B = np.exp((a*d-b*c)/delta)
print('k: ', B, "M: ", A)
data_poblation_col_65_above['y_pred_exp'] = B*(np.exp(A*X))

- Regresión Potencial

In [ ]:
X_log = np.log(X) #Conversion de los datos x en log(x)
a = np.inner(X_log,X_log)
b = np.sum(X_log)
c = np.inner(X_log,Y_log)
d = np.sum(Y_log)
N = len(X)
delta = (a)*(N)-(b**2)
A = (N*c-b*d)/delta
B = np.exp((a*d-b*c)/delta) # Nos determina el k de la función potencial
data_poblation_col_65_above['y_pred_pot'] = B*X**A
print('k: ', B, "M: ", A)
data_poblation_col_65_above

Vamos a ver algunas medidas para ver cual de las regresiones es la más adecuada para nuestros datos, donde:
* `sse`: Suma cuadrática de los error
* `tse`: Varianza de los errores
* `r2`: Relación de las simétricas

Si `r2` es muy cercano a 1, quiere decir que la relacion entre las simétricas es muy buena.

Primero miramemos estos valores para la primera regresión donde tenemos:

In [ ]:
def relation(Y_pred):
    sse = sum((Y-Y_pred)**2)
    tse = (len(Y)-1)*np.var(Y)
    r2 = 1 - sse/tse
    return r2

In [ ]:
print("La relacion entre la regresion lineal y la simetria de los datos es: ", relation(data_poblation_col_65_above['y_pred_lineal']))
print("La relacion entre la regresion exponencial y la simetria de los datos es: ", relation(data_poblation_col_65_above['y_pred_exp']))
print("La relacion entre la regresion potencial y la simetria de los datos es: ", relation(data_poblation_col_65_above['y_pred_pot']))


Podemos ver que las relaciones de la regresión exponencial y potencial tiene casi la misma relación con el ploteo de los datos, por lo tanto en la grafica, estarán sobrexpuestas una de la otra.

In [ ]:
(ggplot(data_poblation_col_65_above, aes(x='year', y='percent'))
+ geom_point(fill="#B100EB",colour="#B100EB")
+ ggtitle('Porcentaje Población x edad >= 64 x Año ')
+ labs(x='Año', y='Porcentaje Población')
+ geom_line(aes( y='y_pred_lineal'), colour="#E61E10")
+ geom_line(aes( y='y_pred_exp'), colour="#14F54A")
+ geom_line(aes( y='y_pred_pot'), colour="#290BFF"))


#### Porcentaje de población total vs. Años para Población con edades de 0 a 14 años.

Realizaremos una regresión lineal y potencial de los porcentajes de edades de 0-14 X Año.
- Regresion lineal: 

In [ ]:
X = data_poblation_col_0_14['year'].values
Y = data_poblation_col_0_14['percent'].values

In [ ]:
a = np.inner(X,X)
b = np.sum(X)
c = np.inner(X,Y)
d = np.sum(Y)
N = len(X)
delta = (a)*(N)-(b**2)
A = (N*c-b*d)/delta
B = (a*d-b*c)/delta
print('B: ', B, "A: ", A)
data_poblation_col_0_14['y_pred_lineal']= A*X+B

- Regresion Potencial

In [ ]:
X_log = np.log(X) #Conversion de los datos x en log(x)
Y_log = np.log(Y) #Conversion de los datos y en log(y)
a = np.inner(X_log,X_log)
b = np.sum(X_log)
c = np.inner(X_log,Y_log)
d = np.sum(Y_log)
N = len(X)
delta = (a)*(N)-(b**2)
A = (N*c-b*d)/delta
B = np.exp((a*d-b*c)/delta) # Nos determina el k de la función potencial
print('k: ', B, "M: ", A)
data_poblation_col_0_14['y_pred_pot'] = B*X**A
data_poblation_col_0_14

In [ ]:
print("La relacion entre la regresion lineal y la simetria de los datos es: ", relation(data_poblation_col_0_14['y_pred_lineal']))
print("La relacion entre la regresion potencial y la simetria de los datos es: ", relation(data_poblation_col_0_14['y_pred_pot']))

In [ ]:
(ggplot(data_poblation_col_0_14, aes(x='year', y='percent'))
+ geom_point(fill="#B100EB",colour="#B100EB")
+ ggtitle('Porcentaje Población x edad 0-14 x Año ')
+ labs(x='Año', y='Porcentaje Población')
+ geom_line(data_poblation_col_0_14, aes(x='year', y='y_pred_lineal'),colour="#9EEB02")
+ geom_line(data_poblation_col_0_14, aes(x='year', y='y_pred_pot'), colour="#EB7413"))

### Expresiones algebraicas de las funciones
Las expreciones de las funciones de prediccion seleccionadas para los datos fueron:
1. Porcentaje de población total vs. Años para Población con edades >= 65 años: 
        $$y = 5.848278004739217\times 10^{-17}e^{0.01953344485763856x}$$ 
2. Porcentaje Población x edad 0-14 x Año:
        $$y = 917.0057724131613 - 0.4426192572125933x$$

Se elijieron en base a los datos de los errores cuadraticos y el valor de relación entre las dos graficas evaluadas anteriormente, igual es facil notar en las tablas obtenidas que tan cerca estan los valores obtenidos en cada regresion con la columna de los porcentajes.

### Use las funciones calculadas en el paso anterior para estimar:

Para este punto hacemos el uso de wolfram para evaluar los siguientes puntos

a. Los años para las poblaciones para los cuales la poblacion con edades >=65 son 50%, 60% y 70%
- Para $$y=50$$, $$x_1\approx 2114$$
- Para $$y=60$$, $$x_1\approx 2123$$
- Para $$y=70$$, $$x_1\approx 2131$$

b. Los años para las poblaciones para los cuales la poblacion con edades entre 0-14 son 20%, 15% y 10%
- Para $$y=20$$, $$x_1\approx 2026$$
- Para $$y=15$$, $$x_1\approx 2038$$
- Para $$y=10$$, $$x_1\approx 2049$$

### Calcule las medidas estadísticas básicas para estas poblaciones. ¿Qué indican cada una de las medidas?

Usaremos la función `describe()` para obtner los medidas basicas para los porcentajes de poblacion
- Porcentaje poblacion >= 65:

In [ ]:
data_poblation_col_65_above['percent'].describe()

De estas medidas estadísticas podemos concluir lo siguiente:

- Se cuentan con 62 registros (1960-2021)

- La media aritmética del porcentaje de personas mayores de 65 años sobre el total de población es de aproximadamente el 5% con una desviación estándar de 1.8%.

- La mediana del porcentaje de personas mayores de 65 años sobre el total de población es del 4.2%, el percentil 25 y 75 son 3.2% y 5.9%

- Debido a que la mediana(4.2%) es mayor a la media(5%) podemos concluir que la distribución de los porcentajes tienen una asimetría hacia la derecha.

- Porcentaje poblacion entre 0-14:

In [ ]:
data_poblation_col_0_14['percent'].describe()

De estas medidas estadísticas podemos concluir lo siguiente:

- Se cuentan con 62 registros (1960-2021)

- La media aritmética del porcentaje de personas menores de 14 años sobre el total de población es de aproximadamente el 35% con una desviación estándar de 8%.

- La mediana del porcentaje de personas menores de 14 años sobre el total de población es del 35%, el percentil 25 y 75 son 29.9% y 43.2%

- Debido a que la mediana y le media toman aproximadamente el mismo valor, podemos decir que la distribución de los porcentajes no presenta un grado de asimetría considerable.

## Ejercicio 2
 
Considere los datos del archivo Excel adjunto; en él se muestran los datos de criminalidad, fondos policiales y educación de la población en las ciudades pequeñas de los Estados Unidos. Las variables `(X1, X2, X3, X4, X5, X6, X7)` representan la siguiente información para cada ciudad:

* `X1` = Reporte total de criminalidad por millón de residentes.
* `X2` = Tasa de crímenes violentos por 100.000 residentes.
* `X3` = Fondos anuales policiales en dólares por habitante.
* `X4` = Porcentaje de personas de 25 años o más con * bachillerato.
* `X5` = Porcentaje de la población de 16 a 19 años sin bachillerato.
* `X6` = Porcentaje de la población entre 18 a 24 años, que realiza estudios universitarios.
* `X7` = Porcentaje de la población de 25 o más años con por lo menos 4 años de estudios universitarios.

Leemos los datos y los guardamos en el DataFrame `criminality_data`:

In [ ]:
criminality_data = pd.read_csv('https://raw.githubusercontent.com/AnalisisDeBasesDatos/TallerEstadBasicaRegMultilineal/main/CriminalidadPequenasCiudades.csv')

### a. Análisis estadístico básico por variable

Vamos a graficar la distribución de cada una de las variables y a calcular algunas estadísticas descriptivas de cada variable:

In [ ]:
fig, axs = plt.subplots(3, 3)

# Ajustar lista de subplots
axs = [ax for axrow in axs for ax in axrow]
axs[-2].axis('off')
axs[-1].axis('off')

for ((name, values), ax) in zip(criminality_data.items(), axs):
    ax.set_title(name)
    ax.hist(values)

fig.tight_layout()
plt.figure(figsize=(16, 7))
plt.show()

Usamos el metodo `describe` para obtener las medidas de centralidad y dispersión de cada variable.

In [ ]:
criminality_data.describe()

### b. Matriz de calor para las correlaciones (de Pearson) entre pares de variables

Calculamos las correlaciones dos-a-dos de cada variable de nuestro conjunto de datos usando el método `corr`.

In [ ]:
pearson_corr = criminality_data.corr()
sns.heatmap(pearson_corr, annot=True)

Como es de esperarse, la matriz es simétrica y la diagonal siempre tiene la misma correlación (igual a 1.0).

### c. Pares de variables con mayor correlación

La gráfica anterior indica que los pares de variables `(X1, X2)` y `(X4, X7)` tienen la mayor correlación positiva, mientras que `(X5, X6)` tienen la mayor correlación negativa. Podemos ordenar esta información en un dataframe:

In [ ]:
def find_most_correlated(df):
    most_correlated = []
    # Por cada variable...
    for colname, col in df.items():
        maxcorr = 0
        varname = ""
        # ...comparamos con cada otra variable
        for rowname, corr in col.items():
            # No nos interesa la correlación de una variable con sí misma
            if colname != rowname:
                # Comparamos valores absolutos, pero guardamos el valor con signo
                if abs(corr) > abs(maxcorr):
                    maxcorr = corr
                    varname = rowname
        most_correlated.append([colname, varname, maxcorr])
    return sorted(most_correlated, key=lambda x: abs(x[2]), reverse=True)

In [ ]:
most_correlated = find_most_correlated(pearson_corr)
pd.DataFrame(most_correlated, columns=["variable1", "variable2", "correlación"])

### d. Análisis multilineal tomando como variable dependiente los fondos anuales policiales con las demás variables como variables predictoras. Indique la expresión que obtuvo.

In [ ]:
# Variables dependientes
X = pd.DataFrame(criminality_data[[col for col in criminality_data.columns if col != 'X3']]).values

# Variable independiente
Y = pd.DataFrame(criminality_data[['X3']]).values

#Generamos una columna de unos y la insertamos como primera columna de X
Unos = np.ones(len(criminality_data))
X = np.insert(X, 0, Unos, axis=1)
MPenrouse = np.linalg.pinv(np.matmul(X.transpose(),X)) # Cálculo de la seudo-inversa de More-Penrose
C = np.matmul(MPenrouse,X.transpose())
B = np.matmul(C,Y)
print(B)

Por lo tanto el modelo se puede expresar mediante la igualdad:
$$
X3 = -4.864 + 0.012\cdot X1 + 0.005\cdot X2 + 0.279\cdot X4 + 0.626\cdot X5 - 0.194\cdot X6 + 0.719\cdot X7
$$

### e. Variables predictoras con mayor impacto sobre la variable independiente

Segun el analisis de regresion que hicimos en el literal d. las variables predictoras con mayor impacto son aquellas con un coeficiente mayor, por lo tanto son las variables **X5 y X7**.

### f. Análisis de regresión lineal simple entre los fondos anuales reales y los predichos por el modelo.

A continuación ajustamos una regresión lineal simple entre los fondos anulaes reales y los predichos:

In [ ]:
Y_pred = np.matmul(X,B)
a = np.inner(Y_pred,Y_pred)
b = np.sum(Y_pred)
c = np.inner(Y_pred,Y)
d = np.sum(Y)
N = len(Y_pred)
Delta = a*N-b*b
A =(N*c-b*d)/Delta
B =(a*d-b*c)/Delta
Y_p = A*Y+B #Y_p es la Y predicha por el modelo de regresión simple
            #Y_pred es la Y predicha por el modelo de regresión múltiple
plt.grid(True)            
plt.scatter(Y,Y_pred, color='red')
plt.plot(Y,Y_p, color='blue')
plt.show()

### g. Modelo de regresión multilineal para los índices criminalidad y el grado de escolaridad de la población

In [ ]:
# Variables independientes
X = pd.DataFrame(criminality_data[['X4', 'X5', 'X6', 'X7']]).values

# Variable dependiente
Y = pd.DataFrame(criminality_data[['X1']]).values

In [ ]:
#Generamos una columna de unos y la insertamos como primera columna de X
ones = np.ones(len(criminality_data))
X = np.insert(X, 0, ones, axis=1)
MPenrouse = np.linalg.pinv(np.matmul(X.transpose(),X)) # Cálculo de la seudo-inversa de More-Penrose
C = np.matmul(MPenrouse,X.transpose())
B = np.matmul(C,Y)
print(B)

Por lo tanto el modelo se puede expresar mediante la igualdad:
$$
X1 = 606.713 - 5.102\cdot X4 + 15.124\cdot X5 - 2.987\cdot X6 + 19.367\cdot X7
$$

### h. ¿Es posible detectar datos atípicos en la base de datos?

In [ ]:
criminality_data.iqr()

In [ ]:
sns.boxplot(data=criminality_data)
matplotlib.pyplot.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a48fd26b-8f06-44e6-b375-918c7d927261' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>